In [2]:
pip install scapy pyshark

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
from scapy.all import sniff, TCP, IP, UDP, DNS

In [ ]:
#Bacis of how to callback a packet information

def pkt_callback(packet):
    packet.show()

sniff(prn= pkt_callback, count= 3)

###[ Ethernet ]###
  dst       = d8:07:b6:ec:33:a2
  src       = 0e:11:78:36:0a:ba
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 170
     id        = 0
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0xba68
     src       = 192.168.0.118
     dst       = 40.79.150.120
     \options   \
###[ TCP ]###
        sport     = 51620
        dport     = https
        seq       = 1636650060
        ack       = 3535114702
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 2048
        chksum    = 0x469a
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (4040933642, 226530711))]
###[ Raw ]###
           load      = b'\x17\x03\x03\x00q\xc5\xf0\x94G<UOP\xa4Q`\n\x93\xe6\xf92P\xd3\xd2\x91\x9a\x87<\xad\x8d\xd9\xacA\xbd\xa9I\x8e)h\xac\xcf\xb3"=\xf7s\xa2Yd\x109\x8e\xccT\x13$\xe8\xcb\x08T\x86\xb4\x83\xf1\xb1\x06f\x85\xc9o\x9

<Sniffed: TCP:3 UDP:0 ICMP:0 Other:0>

In [8]:
#how to single out key information and dispalying it. 

def process_pkt(packet):
    if packet.haslayer(IP):
        protocol = "TCP" if packet.haslayer(TCP) else "UDP" if packet.haslayer(UDP) else "Other"
        print(f"Packet :{packet[IP].src} -> {packet[IP].dst} | Protocol:{protocol}")

sniff(prn=process_pkt,count=10)

Packet :192.168.0.118 -> 224.0.0.251 | Protocol:UDP
Packet :192.168.0.118 -> 172.224.35.135 | Protocol:UDP
Packet :192.168.0.118 -> 172.224.35.135 | Protocol:UDP
Packet :192.168.0.118 -> 172.224.35.135 | Protocol:UDP
Packet :172.224.35.135 -> 192.168.0.118 | Protocol:UDP
Packet :172.224.35.135 -> 192.168.0.118 | Protocol:UDP
Packet :172.224.35.135 -> 192.168.0.118 | Protocol:UDP


<Sniffed: TCP:0 UDP:9 ICMP:0 Other:1>

In [15]:
def dns_callback(packet):
    if packet.haslayer(DNS):
        print(f"DNS Request: {packet[DNS].qd.qname.decode()}")


sniff(filter="udp port 53", prn=dns_callback, count=60)

DNS Request: www.youtube.com.
DNS Request: www.youtube.com.
DNS Request: www.youtube.com.
DNS Request: www.youtube.com.
DNS Request: i.ytimg.com.
DNS Request: i.ytimg.com.
DNS Request: i.ytimg.com.
DNS Request: i.ytimg.com.
DNS Request: www.google.com.
DNS Request: www.google.com.
DNS Request: www.google.com.
DNS Request: www.google.com.
DNS Request: rr5---sn-5hne6ns6.googlevideo.com.
DNS Request: rr5---sn-5hne6ns6.googlevideo.com.
DNS Request: rr5---sn-5hne6ns6.googlevideo.com.
DNS Request: rr5---sn-5hne6ns6.googlevideo.com.
DNS Request: www.gstatic.com.
DNS Request: www.gstatic.com.
DNS Request: www.gstatic.com.
DNS Request: www.gstatic.com.
DNS Request: accounts.youtube.com.
DNS Request: accounts.youtube.com.
DNS Request: accounts.youtube.com.
DNS Request: accounts.youtube.com.
DNS Request: rr1---sn-5jucgv5qc5oq-cagk.googlevideo.com.
DNS Request: rr1---sn-5jucgv5qc5oq-cagk.googlevideo.com.
DNS Request: rr1---sn-5jucgv5qc5oq-cagk.googlevideo.com.
DNS Request: rr1---sn-5jucgv5qc5oq-ca

<Sniffed: TCP:0 UDP:60 ICMP:0 Other:0>

In [18]:
failed_logins = {}

def detect_bruteforce(packet):
    if packet.haslayer(TCP) and packet[TCP].flags == 2:
        src_ip = packet[IP].src
        print(packet[IP].src,"->" ,packet[IP].dst)
        failed_logins[src_ip]= failed_logins.get(src_ip,0) + 1
        if failed_logins[src_ip] > 5:
            print(f"Potential brute-force attack from {src_ip}")

sniff(filter='tcp', prn=detect_bruteforce, count= 10)

<Sniffed: TCP:10 UDP:0 ICMP:0 Other:0>